In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Clust_Prjt').getOrCreate()

In [2]:
dataset = spark.read.csv('Clustering/hack_data.csv',inferSchema=True,header=True)

In [3]:
dataset.head(1)

[Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37)]

In [4]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler

In [5]:
dataset.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [6]:
feat_cols = ['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'WPM_Typing_Speed']

In [7]:
assembler = VectorAssembler(inputCols=feat_cols,outputCol='features')

In [8]:
final_data = assembler.transform(dataset)

In [9]:
final_data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- features: vector (nullable = true)



In [10]:
from pyspark.ml.feature import StandardScaler

In [15]:
# Creating object for the scaler
scaler = StandardScaler (inputCol='features', outputCol='scaledFeature')

In [16]:
# Creating scaler model by fitting as it like the machine learning model
scaler_model = scaler.fit(final_data)

In [17]:
cluster_final_data = scaler_model.transform(final_data)

In [18]:
cluster_final_data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- scaledFeature: vector (nullable = true)



In [19]:
# Creating kmeans model
kmeans2 = KMeans(featuresCol='scaledFeature',k=2)
kmeans3 = KMeans(featuresCol='scaledFeature',k=3)

In [20]:
model_k2 = kmeans2.fit(cluster_final_data)
model_k3 = kmeans3.fit(cluster_final_data)

In [21]:
# Evaluating the model by transforming the data and finding the answer
model_k3.transform(cluster_final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   84|
|         2|   83|
|         0|  167|
+----------+-----+



In [22]:
# Evaluating the model by transforming the data and finding the answer
model_k2.transform(cluster_final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+

